In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load the dataset
df = pd.read_csv('plant_growth_data_c72367ea.csv')

# Remove the time attribute
df = df.drop(columns=['time'])

# Split the dataset into input (X) and output (y) variables
X = df[['temperature_2m', 'relativehumidity_2m']].values
y = df[['rain', 'soil_temperature_0_to_7cm', 'soil_moisture']].values

# Scale the data
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Reshape the input data into 3D tensor with shape (n_samples, timesteps, n_features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(1, 2)))
model.add(Dense(units=3))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Make predictions on new data
X_new = np.array([[25, 60]])  # Example new data
X_new_scaled = scaler_X.transform(X_new)
X_new_reshaped = X_new_scaled.reshape((X_new_scaled.shape[0], 1, X_new_scaled.shape[1]))
y_new_scaled = model.predict(X_new_reshaped)
y_new = scaler_y.inverse_transform(y_new_scaled)
print(y_new)


Epoch 1/50
250/250 [==============================] - 3s 4ms/step - loss: 0.1297 - val_loss: 0.0889
Epoch 2/50
250/250 [==============================] - 1s 3ms/step - loss: 0.0853 - val_loss: 0.0856
Epoch 3/50
250/250 [==============================] - 1s 3ms/step - loss: 0.0831 - val_loss: 0.0850
Epoch 4/50
250/250 [==============================] - 1s 3ms/step - loss: 0.0827 - val_loss: 0.0853
Epoch 5/50
250/250 [==============================] - 1s 3ms/step - loss: 0.0826 - val_loss: 0.0851
Epoch 6/50
250/250 [==============================] - 1s 3ms/step - loss: 0.0826 - val_loss: 0.0850
Epoch 7/50
250/250 [==============================] - 1s 3ms/step - loss: 0.0826 - val_loss: 0.0851
Epoch 8/50
250/250 [==============================] - 1s 3ms/step - loss: 0.0826 - val_loss: 0.0849
Epoch 9/50
250/250 [==============================] - 1s 3ms/step - loss: 0.0827 - val_loss: 0.0852
Epoch 10/50
250/250 [==============================] - 1s 3ms/step - loss: 0.0826 - val_loss: 0.0851

In [2]:
#save model
model.save('lstm.h5')


In [3]:
#check model paramaters
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10600     
                                                                 
 dense (Dense)               (None, 3)                 153       
                                                                 
Total params: 10,753
Trainable params: 10,753
Non-trainable params: 0
_________________________________________________________________
None


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model

# Load the test dataset
test_df = pd.read_csv('plant_growth_data_c72367ea.csv')

# Remove the time attribute
test_df = test_df.drop(columns=['time'])

# Split the dataset into input (X) and output (y) variables
X_test = test_df[['temperature_2m', 'relativehumidity_2m']].values
y_test = test_df[['rain', 'soil_temperature_0_to_7cm', 'soil_moisture']].values

# Scale the test data using the same scaler used for the training data
scaler_X = MinMaxScaler()
X_test_scaled = scaler_X.fit_transform(X_test)

scaler_y = MinMaxScaler()
y_test_scaled = scaler_y.fit_transform(y_test)

# Reshape the input data into 3D tensor with shape (n_samples, timesteps, n_features)
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Load the saved model
model = load_model('D:\my_project\models\plant_growth_data_d23f1206_best.h5')

# Use the model to predict the output for the test dataset
y_pred_scaled = model.predict(X_test_reshaped)

# Inverse transform the scaled output to get the actual values
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Calculate the percentage of correct predictions for each output variable
accuracy_rain = 100 - np.mean(np.abs((y_pred[:,0] - y_test[:,0]) / y_test[:,0])) * 100
accuracy_soil_temp = 100 - np.mean(np.abs((y_pred[:,1] - y_test[:,1]) / y_test[:,1])) * 100
accuracy_soil_moisture = 100 - np.mean(np.abs((y_pred[:,2] - y_test[:,2]) / y_test[:,2])) * 100

# Print the accuracy of the model on the test dataset
print('Accuracy - Rain: %.2f%%' % accuracy_rain)
print('Accuracy - Soil Temperature: %.2f%%' % accuracy_soil_temp)
print('Accuracy - Soil Moisture: %.2f%%' % accuracy_soil_moisture)


313/313 [==============================] - 2s 1ms/step
Accuracy - Rain: 89.86%
Accuracy - Soil Temperature: 96.78%
Accuracy - Soil Moisture: 99.08%


In [2]:
#take two inputs and predict three outputs using pretrained model
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model

# take two inputs
X_new = np.array([[25, 60]])  # Example new data

# Load the saved model
model = load_model('lstm.h5')

# Scale the new data using the same scaler used for the training data
scaler_X = MinMaxScaler()
X_new_scaled = scaler_X.fit_transform(X_new)

# Reshape the input data into 3D tensor with shape (n_samples, timesteps, n_features)
X_new_reshaped = X_new_scaled.reshape((X_new_scaled.shape[0], 1, X_new_scaled.shape[1]))

# Use the model to predict the output for the new data
y_new_scaled = model.predict(X_new_reshaped)

# Inverse transform the scaled output to get the actual values
y_new = scaler_y.inverse_transform(y_new_scaled)

# Print the predicted output
print(y_new)


1/1 [==============================] - 0s 201ms/step
[[ 2.5087328 23.478815  54.00847  ]]


In [ ]:
#take two inputs and predict three outputs using pretrained model, using tkinter create a gui window with browse model button and two fields to input data and pridict the output and show in the window
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model

# Create a window
window = tk.Tk()

# Function for browsing the model file
def browse_model():
    global model_path
    model_path = filedialog.askopenfilename()
    model_entry.delete(0, tk.END)
    model_entry.insert(0, model_path)

# Function for predicting the output
def predict_output():
    # Load the saved model
    model = load_model(model_path)

    # Take the inputs
    X_new = np.array([[float(temperature_entry.get()), float(humidity_entry.get())]])

    # Scale the new data using the same scaler used for the training data
    scaler_X = MinMaxScaler()
    X_new_scaled = scaler_X.fit_transform(X_new)

    # Reshape the input data into 3D tensor with shape (n_samples, timesteps, n_features)
    X_new_reshaped = X_new_scaled.reshape((X_new_scaled.shape[0], 1, X_new_scaled.shape[1]))

    # Use the model to predict the output for the new data
    y_new_scaled = model.predict(X_new_reshaped)

    # Inverse transform the scaled output to get the actual values
    y_new = scaler_y.inverse_transform(y_new_scaled)

    # Print the predicted output
    rain_entry.delete(0, tk.END)
    rain_entry.insert(0, y_new[0][0])
    soil_temp_entry.delete(0, tk.END)
    soil_temp_entry.insert(0, y_new[0][1])
    soil_moisture_entry.delete(0, tk.END)
    soil_moisture_entry.insert(0, y_new[0][2])

# Create a label for the model path
model_label = tk.Label(window, text='Model Path:')
model_label.grid(row=0, column=0, padx=5, pady=5)

# Create a entry for the model path
model_entry = tk.Entry(window, width=50)
model_entry.grid(row=0, column=1, padx=5, pady=5)

# Create a button for browsing the model file
browse_button = tk.Button(window, text='Browse', command=browse_model)
browse_button.grid(row=0, column=2, padx=5, pady=5)

# Create a label for temperature
temperature_label = tk.Label(window, text='Temperature:')
temperature_label.grid(row=1, column=0, padx=5, pady=5)

# Create a entry for temperature
temperature_entry = tk.Entry(window, width=50)
temperature_entry.grid(row=1, column=1, padx=5, pady=5)

# Create a label for humidity
humidity_label = tk.Label(window, text='Humidity:')
humidity_label.grid(row=2, column=0, padx=5, pady=5)

# Create a entry for humidity
humidity_entry = tk.Entry(window, width=50)
humidity_entry.grid(row=2, column=1, padx=5, pady=5)

# Create a button for predicting the output
predict_button = tk.Button(window, text='Predict', command=predict_output)
predict_button.grid(row=3, column=1, padx=5, pady=5)

# Create a label for rain
rain_label = tk.Label(window, text='Rain:')
rain_label.grid(row=4, column=0, padx=5, pady=5)

# Create a entry for rain
rain_entry = tk.Entry(window, width=50)
rain_entry.grid(row=4, column=1, padx=5, pady=5)

# Create a label for soil temperature
soil_temp_label = tk.Label(window, text='Soil Temperature:')
soil_temp_label.grid(row=5, column=0, padx=5, pady=5)

# Create a entry for soil temperature
soil_temp_entry = tk.Entry(window, width=50)
soil_temp_entry.grid(row=5, column=1, padx=5, pady=5)

# Create a label for soil moisture
soil_moisture_label = tk.Label(window, text='Soil Moisture:')
soil_moisture_label.grid(row=6, column=0, padx=5, pady=5)

# Create a entry for soil moisture
soil_moisture_entry = tk.Entry(window, width=50)
soil_moisture_entry.grid(row=6, column=1, padx=5, pady=5)

# Start the window
window.mainloop()
